# Exoplanet Predictions
*By Adam Zheng*

> ### NOTE:
> Predictions from this program are most likely not accurate enough to be used for scientific purposes. The data used in the training of this model is indeed sourced from __[NASA Exoplanet Archive's data on Kepler Objects of Interest (KOIs)](https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=cumulative)__. And I was able to obtain a 91.52% accuracy rate when testing this model on data it hadn't encountered in training (*see koi_models). However, it was not thoroughly or professionally vetted for potential issues, and the training was only done using 13 features, so these results may not reflect the actual accuracy of the model. In the end, this program was created largely just as a way for me to practice machine learning in Python. As such, this program should be taken with a grain (or perhaps a bucketload) of salt.

#### The options for entering data input are as follows:
1. Enter a KOI Name for a KOI candidate that the model has already predicted (should work for almost all KOI candidates)
2. Make predictions using your own CSV file with data values
3. Manually enter data values for each of your entries

##### The required features are (for options 2 & 3):
1. Orbital Period (days)
2. Transit Epoch (BJD - 2,454,833.0)
3. Impact Parameter
4. Transit Duration (hours)
5. Transit Depth (parts per million)
6. Planetary Radius (Earth radii)
7. Equilibrium Temperature (Kelvin)
8. Transit Signal-to-Noise
9. Stellar Effective Temperature (Kelvin)
10. Stellar Radius (solar radii)
11. Right Ascension (deg)
12. Declination (deg)
13. Kepler-band (mag)
>*For more detailed information on the features: https://exoplanetarchive.ipac.caltech.edu/docs/API_kepcandidate_columns.html*

In [ ]:
# MUST RUN THIS CODE
import joblib
import numpy as np
import pandas as pd
import warnings
import time
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

koi_model = joblib.load('model_training/GB_koi_model.pkl')

### 1) Find Existing KOI Candidates

In [ ]:
cand_pred = pd.read_csv("koi_candidate_predictions.csv")
valid_input = False
while not valid_input:
    try:
        cand = input("Enter a KOI Name: ").strip()
        if len(cand) == 9 and cand[0]=='K':
            valid_input = True
        else:
            print("\tERROR: PLEASE ENTER A VALID KOI NAME (ex. K00760.01)")
    except:
        print("\tERROR: PLEASE ENTER A VALID KOI NAME (ex. K00760.01)")
if cand in cand_pred["KOI Name"].unique():
    ind = cand_pred[cand_pred['KOI Name']==cand].index.values[0]
    print("\n- SEARCH SUCCESSFUL -")
    print(f'{cand} was predicted as {cand_pred.loc[ind].at["Exoplanet Prediction"]}')
else:
    print(f"{cand} was not found in our list of existing KOI candidates.")

### 2) Use Your CSV File

*PLEASE UPLOAD YOUR CSV FILE TO THE SAME FOLDER AS THIS PREDICTOR PROGRAM BEFORE PROCEEDING.*

In [ ]:
valid_input = False
while not valid_input:
    try:
        csv_name = input("Enter the file name of your CSV file: ").strip()
        data = pd.read_csv(csv_name)
        valid_input = True
    except:
        print("\tERROR: PLEASE ENTER A VALID CSV FILE NAME")
predictions = pd.DataFrame(koi_model.predict(data), columns=["Prediction"])
compiled_predictions = pd.concat([predictions, data], axis=1)
compiled_predictions.to_csv('YOUR_PREDICTIONS.csv', index=False)
print("\n- PREDICTIONS COMPLETED -")
print("The prediction results have also been exported as a CSV file called YOUR_PREDICTIONS.csv")
compiled_predictions

### 3) Manual Input

In [ ]:
from IPython.display import display, HTML

model_params = ['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_model_snr', 
                'koi_steff', 'koi_srad', 'ra', 'dec', 'koi_kepmag']
display_params = ['Orbital Period (days)', 'Transit Epoch (BJD - 2,454,833.0)', 'Impact Parameter', 'Transit Duration (hours)', 
                  'Transit Depth (parts per million)', 'Planetary Radius (Earth radii)', 'Equilibrium Temperature (Kelvin)', 
                  'Transit Signal-to-Noise', 'Stellar Effective Temperature (Kelvin)', 'Stellar Radius (solar radii)', 
                  'Right Ascension (deg)', 'Declination (deg)', 'Kepler-band (mag)']
test_params = [1.0]*13 # purely for testing purposes
input_vals = [[0]*13]

# USER INPUT
valid_input = False
stop = False
while not valid_input:
    try:
        num_inputs = int(input("How many entries would you like to make? ").strip())
        if num_inputs >= 1:
            valid_input = True
        else:
            print("\tERROR: PLEASE ENTER A POSITIVE INTEGER ")
    except:
        print("\tERROR: PLEASE ENTER A POSITIVE INTEGER ")
print("  *If at any point, you would like to terminate the program, simply enter 'stop'")
for entry in range(num_inputs):
    if stop:
        break
    temp_list = []
    print(f"\n--- ENTRY #{entry+1} ---")
    for i in range(len(display_params)):
        if stop:
            break
        valid_input = False
        while not valid_input:
            if stop:
                break
            try:
                temp = input(f'Enter the {display_params[i]}: ').strip()
                if temp in ["stop", "'stop'"]:
                    stop = True
                    break
                temp_list.append(float(temp))
                #temp_list.append(test_params[i])
                valid_input = True
            except:
                print("\tERROR: PLEASE ENTER A NUMBER")
    input_vals.insert(entry, temp_list)

# RUN MODEL
if not stop:
    params = pd.DataFrame(np.array(input_vals), columns=model_params)
    params.drop(index=num_inputs, inplace=True)
    pred = koi_model.predict(params)
    print("\n--- PREDICTIONS ---")
    for i in range(len(pred)):
        print(f'{i+1}) {pred[i]}')
    print("\n--- FULL DATA TABLE ---")
    predictions = pd.DataFrame(pred, columns=["Prediction"])
    compiled_predictions = pd.concat([predictions, params], axis=1)
    compiled_predictions.rename(columns=dict(zip(model_params, display_params)), inplace=True)
    display(compiled_predictions)
    compiled_predictions.to_csv('YOUR_PREDICTIONS.csv', index=False)
    print("The prediction results have been exported as a CSV file called YOUR_PREDICTIONS.csv")
else:
    print("PROGRAM ENDED")